In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\doc-bot'

In [22]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings


In [46]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [47]:
extracted_data=load_pdf_file(data='Data/')

In [51]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunk=text_splitter.split_documents(extracted_data)
    return text_chunk

In [52]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks",len(text_chunks))

Length of Text Chunks 5860


In [54]:

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [55]:
embeddings =download_hugging_face_embeddings()

In [26]:
query_result = embeddings.embed_query("hello world")
print("Length",len(query_result))

Length 384


In [28]:
#query_result

In [86]:
from dotenv import load_dotenv
load_dotenv()

True

In [87]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
index_name = "doc-bot"

pc =Pinecone(api_key=PINECONE_API_KEY)
pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [89]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch =PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [61]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [67]:
from langchain_groq import ChatGroq

In [91]:
llm = ChatGroq(
    api_key="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",
    model_name="llama3-70b-8192",  # You can choose different models like "mixtral-8x7b" or "llama3-8b"
    temperature=0.4,
    max_tokens=500,
    
)

In [92]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrived context to answer"
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer conscise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [93]:
retriever = docsearch.as_retriever(
    search_type="similarity",  # You can use "similarity", "mmr", or other search types
    search_kwargs={"k": 3}  # Return top 4 most relevant documents
)

# Now create your chains
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [94]:
response = rag_chain.invoke({"input":"What is Acne?"})
print(response["answer"])

Acne is a common skin disease characterized by pimples on the face, chest, and back, occurring when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
